In [ ]:
print("OK!")

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [ ]:
extracted_data=load_pdf_file(data='data/')

In [ ]:
extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
    texts = text_splitter.split_documents(extracted_data)
    return texts

In [ ]:
texts=text_split(extracted_data)
print(f"Total number of chunks: {len(texts)}")  

Total number of chunks: 9826


In [ ]:
texts

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:

embeddings=download_hugging_face_embeddings()

In [ ]:
query_result=embeddings.embed_query("Hello world")
print("length",len(query_result))

In [ ]:
query_result

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_4UboeU_JB8iqzdQjRuW8aDhH8CSRTaPkjcnLqbHZypRLx3mLoF46QUkqFUMgTqAtACxxxA")

index_name = "testbot"

# ✅ Check if index already exists
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"✅ Index '{index_name}' created.")
else:
    print(f"ℹ️ Index '{index_name}' already exists.")



In [ ]:
index = pc.Index(index_name)


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_4UboeU_JB8iqzdQjRuW8aDhH8CSRTaPkjcnLqbHZypRLx3mLoF46QUkqFUMgTqAtACxxxA"

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=texts,
    index_name=index_name,
    embedding=embeddings,
)

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever_docs=retriever.invoke("What is acne?")

In [ ]:
retriever_docs

In [ ]:
prompt_template="""
Use the following pieces of information to answer the users's question
if you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}

Only answer the question based on the context provided.
"""


In [ ]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}


In [ ]:
from langchain.llms import CTransformers

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # ✅ Full path to your GGML file
    model_type="llama",                             # ✅ Model type
    config={
        'max_new_tokens': 512,
        'temperature': 0.8,
        'top_k': 50,
        'top_p': 0.95,
        'repetition_penalty': 1.1
    }
)

# Generate output
prompt = "Explain the concept of machine learning."
output = llm(prompt)
print(output)


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA


# Create vectorstore (docsearch)
docsearch = FAISS.from_documents(texts, embeddings)

# ✅ Confirm it's a FAISS object
print("Type of docsearch:", type(docsearch))

# Get retriever
retriever = docsearch.as_retriever(search_kwargs={"k": 1})

# Optional chain settings
chain_type_kwargs = {"verbose": True}

# ✅ Build RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)




In [ ]:
while True:
    user_input = input("Ask a question (or type 'exit' to quit): ").strip()

    if user_input.lower() in ["exit", "quit", "q"]:
        print("👋 Goodbye!")
        break

    result = qa({"query": user_input})
    print("\nAnswer:", result["result"])


In [64]:
print("\nAnswer:", result["result"])